In [3]:
class mySeq2seq():
    def __init__(self,X,Y,batch_size = 50,hidden_dim = 35,layers_stacked_count = 2,learning_rate = 0.007,lr_decay = 0.92,momentum = 0.5,lambda_l2_reg = 0.003):
        seq_length = X[0]  # Time series will have the same past and future (to be predicted) lenght. 
        output_dim = Y[-1]
        input_dim = X[-1]  # Output dimension (e.g.: multiple signals at once, tied in time)
        try:
            tf.nn.seq2seq = tf.contrib.legacy_seq2seq
            tf.nn.rnn_cell = tf.contrib.rnn
            tf.nn.rnn_cell.GRUCell = tf.contrib.rnn.GRUCell
            print("TensorFlow's version : 1.0 (or more)")
        except: 
            print("TensorFlow's version : 0.12")
        tf.reset_default_graph()
        # sess.close()
        sess = tf.InteractiveSession()
        with tf.variable_scope('Seq2seq'):
            enc_inp = [
                tf.placeholder(tf.float32, shape=(None, input_dim), name="inp_{}".format(t))
                   for t in range(seq_length)
            ]
            expected_sparse_output = [
                tf.placeholder(tf.float32, shape=(None, output_dim), name="expected_sparse_output_".format(t))
                  for t in range(seq_length)
            ]
            dec_inp = [ tf.zeros_like(enc_inp[0], dtype=np.float32, name="GO") ] + enc_inp[:-1]
            cells = []
            for i in range(layers_stacked_count):
                with tf.variable_scope('RNN_{}'.format(i)):
                    cells.append(tf.nn.rnn_cell.GRUCell(hidden_dim))
                    # cells.append(tf.nn.rnn_cell.BasicLSTMCell(...))
            cell = tf.nn.rnn_cell.MultiRNNCell(cells)
            dec_outputs, dec_memory = tf.nn.seq2seq.basic_rnn_seq2seq(
                enc_inp, 
                dec_inp, 
                cell
            )
            w_out = tf.Variable(tf.random_normal([hidden_dim, output_dim]))
            b_out = tf.Variable(tf.random_normal([output_dim]))
            output_scale_factor = tf.Variable(1.0, name="Output_ScaleFactor")
            reshaped_outputs = [output_scale_factor*(tf.matmul(i, w_out) + b_out) for i in dec_outputs]
        with tf.variable_scope('Loss'):
            # L2 loss
            output_loss = 0
            for _y, _Y in zip(reshaped_outputs, expected_sparse_output):
                output_loss += tf.reduce_mean(tf.nn.l2_loss(_y - _Y))

            # L2 regularization (to avoid overfitting and to have a  better generalization capacity)
            reg_loss = 0
            for tf_var in tf.trainable_variables():
                if not ("Bias" in tf_var.name or "Output_" in tf_var.name):
                    reg_loss += tf.reduce_mean(tf.nn.l2_loss(tf_var))

            loss = output_loss + lambda_l2_reg * reg_loss

        with tf.variable_scope('Optimizer'):
            optimizer = tf.train.RMSPropOptimizer(learning_rate, decay=lr_decay, momentum=momentum)
            train_op = optimizer.minimize(loss)
    def train(batch_size,nb_iters,TrainX,TrainY,TestX,TestY):
        train_losses = []
        test_losses = []
        sess.run(tf.global_variables_initializer())
        for t in range(nb_iters+1):
            train_loss = train_batch(batch_size,TrainX,TrainY)
            train_losses.append(train_loss)

            if t % 10 == 0: 
                # Tester
                test_loss = test_batch(batch_size,TestX,TestY)
                test_losses.append(test_loss)
                print("Step {}/{}, train loss: {}, \tTEST loss: {}".format(t, nb_iters, train_loss, test_loss))

        print("Fin. train loss: {}, \tTEST loss: {}".format(train_loss, test_loss))
    def train_batch(batch_size,X,Y):
        """
        Training step that optimizes the weights 
        provided some batch_size X and Y examples from the dataset. 
        """
        feed_dict = {enc_inp[t]: X[t] for t in range(len(X))}
        feed_dict.update({expected_sparse_output[t]: Y[t] for t in range(len(Y))})
        _, loss_t = sess.run([train_op, loss], feed_dict)
        return loss_t

    def test_batch(batch_size):
        """
        Test step, does NOT optimizes. Weights are frozen by not
        doing sess.run on the train_op. 
        """
        feed_dict = {enc_inp[t]: X[t] for t in range(len(X))}
        feed_dict.update({expected_sparse_output[t]: Y[t] for t in range(len(Y))})
        loss_t = sess.run([loss], feed_dict)
        return loss_t[0]